# Implementar un Multilayer Perceptron  con PyTorch para clasificación de texto
# Uso de word embeddings como representación de datos


<img src="figs/fig-diagrama-clasificador2.png" width="900">


### 1. **Representar los datos en el modelo de word embeddings seleccionado**:  
   - #### Generalmente, solo se tokeniza para separar adecuadamente las palabras.
   - #### Sin embargo, dependiendo del modelos de word embeddings algunos preprocesamientos puede mejorar la representación.
   - #### Por ejemplo: 
      - ##### tokenizar y separar correctamente las oraciones y palabras
      - ##### convertir a minúsculas
      - ##### quitar acentos (dependiendo de la fuente de datos con la que se generaros los embeddings)
      - ##### quitar números y puntuación 


### 3. **Convertir los datos a vectores densos: word embeddings**:  
   - #### En el caso de textos corto a nivel de oración, un vector denso por oración. 

### 4. **Separar los datos para entrenamiento, validación y prueba**:  
   - #### Crear los dataset  con la función train_test_split 
   
### 5. **Definir la arquitectura de la red**:  
   - Definir una red de 2 capas, con funciones PReLU en las capas ocultas y una capa de salida

### 6. **Entrenar el modelo**:  
   - Definir los parámetros de las red como: número de épocas, learning_rate, número de neuronas para las capas ocultas, etc.
   
### 7. **Evaluar el modelo**:  
   - Después del entrenamiento, probar la red con las entradas del conjunto de test y evaluar el desempeño con las métricas: Precisión, Recall, F1-score o F1-Measure y Accuracy.
   

### 8. **Guardar el modelo**:  
   - Después del entrenamiento, el modelo se guarda para posteriores usos: despliegue del modelo para aplicación web, o como punto de partida para optimizar por medio de Computación Evolutiva.
   

# Cargar los datos de entrenamiento

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# colocar la semilla para la generación de números aleatorios para la reproducibilidad de experimentos

random_state = 42
torch.manual_seed(random_state)
np.random.seed(random_state)

#cargar los datos
dataset_train = pd.read_json("./data/dataset_polaridad_es_train_embeddings.json", lines=True)
dataset_test = pd.read_json("./data/dataset_polaridad_es_test_embeddings.json", lines=True)

#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset_train.klass.value_counts())
# Extracción de los vectores de embeddings que representan a los textos
X = np.vstack(dataset_train['we_es'].to_numpy())
# Extracción de las etiquetas o clases de entrenamiento
Y = dataset_train['klass'].to_numpy()

X_test = np.vstack(dataset_test['we_es'].to_numpy())
# Extracción de las etiquetas o clases de entrenamiento
Y_test = dataset_test['klass'].to_numpy()


print("Datos train:", X.shape) 
print("Etiquetas train:", Y.shape) 
print("Datos test:", X_test.shape) 
print("Etiquetas test:", Y_test.shape) 



Total de ejemplos de entrenamiento
klass
neutral     1485
positive     968
negative     689
Name: count, dtype: int64
Datos train: (3142, 300)
Etiquetas train: (3142,)
Datos test: (786, 300)
Etiquetas test: (786,)


# Codificar las etiquetas

In [43]:
# TODO: Codificar las etiquetas de los datos a una forma categórica numérica: LabelEncoder.

le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_train_encoded= le.fit_transform(Y)
print("Clases:")
print(le.classes_)
print("Clases codificadas:")
print(le.transform(le.classes_))


Clases:
['negative' 'neutral' 'positive']
Clases codificadas:
[0 1 2]


# Preparar los conjuntos de datos  para el entrenamiento, validación y prueba

In [44]:

# Dividir el conjunto de entrenamiento en:  entrenamiento (80%) y validación (10%)
X_train, X_val, Y_train, Y_val =  train_test_split(X, Y_train_encoded, test_size=0.1, stratify=Y_train_encoded, random_state=42)



# Codificar las clases en forma one-hot 
NUM_CLASSES = 3
Y_train_one_hot = nn.functional.one_hot(torch.from_numpy(Y_train), num_classes=NUM_CLASSES).float()


# Crear minibatches en PyTorch usando DataLoader
def create_minibatches(X, Y, batch_size):
    # Recibe los documentos en X y las etiquetas en Y
    dataset = TensorDataset(X, Y) # Cargar los datos en un dataset de tensores
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # loader = DataLoader(dataset, batch_size=batch_size)
    return loader


In [45]:
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

((2827, 300), (2827,), (315, 300), (315,), (786, 300), (786,))

In [46]:
Y_train_one_hot[:5],  Y_train_one_hot.shape

(tensor([[0., 1., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]),
 torch.Size([2827, 3]))

# Definición de la arquitectura de la red

In [47]:

# Definir la red neuronal en PyTorch heredando de la clase base de Redes Neuronales: Module
class RedNeuronal(nn.Module):
    def __init__(self, tam_entrada, tam_salida):
        super().__init__()
        # Definición de capas, funciones de activación e inicialización de pesos
        tam_entrada_capa_oculta_1 = 128
        tam_entrada_capa_oculta_2 = 8 
        self.fc1 = nn.Linear(tam_entrada, tam_entrada_capa_oculta_1)
        self.fc2 = nn.Linear(tam_entrada_capa_oculta_1, tam_entrada_capa_oculta_2)
        self.salida = nn.Linear(tam_entrada_capa_oculta_2, tam_salida)
        self.activacion= nn.PReLU()

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.salida.weight)

        if self.fc1.bias is not None:
            nn.init.zeros_(self.fc1.bias)
        if self.fc2.bias is not None:
            nn.init.zeros_(self.fc2.bias)        
        if self.salida.bias is not None:
            nn.init.zeros_(self.salida.bias)        

    
    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        x = self.fc1(X)
        x = self.activacion(x)
        x = self.fc2(x)
        x = self.activacion(x)
        x = self.salida(x)
        return x

# Entrenamiento de la red

In [48]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Establecer los parámetros de la red

# Parámetros de la red
tam_entrada =  X_train.shape[1]

tam_salida = 3   # 3 clases

epochs = 30 # variar el número de épocas, para probar que funciona la programación 
                 # solo usar 2 épocas, para entrenamiento total usar por ejemplo 1000 épocas
learning_rate = 0.001 # Generalmente se usan learning rate pequeños (0.001), 

# Se recomiendan tamaños de batch_size potencias de 2: 16, 32, 64, 128, 256
# Entre mayor el número más cantidad de memoria se requiere para el procesamiento
batch_size = 128 # definir el tamaño del lote de procesamiento 


# Convertir los datos de entrenamiento y etiquetas a tensores  de PyTorch

X_tensor_train = torch.from_numpy(X_train)
X_tensor_train = X_tensor_train.to(torch.float32)


X_tensor_val = torch.from_numpy(X_val)
X_tensor_val = X_tensor_val.to(torch.float32)


# Crear la red
modelo_red_neuronal = RedNeuronal(tam_entrada, tam_salida)

# Definir la función de pérdida
# Entropía Cruzada 
criterion = nn.CrossEntropyLoss() 

# Definir el optimizador
# Parámetros del optimizador: parámetros del modelo y learning rate 
# Adaptive Moment Estimation
optimizer = optim.Adam(modelo_red_neuronal.parameters(), lr=learning_rate)

# Entrenamiento
print("Iniciando entrenamiento en PyTorch")

for epoch in range(epochs):
# Poner el modelo en modo de entrenamiento
    modelo_red_neuronal.train()  
    lossTotal = 0
    #definir el batch_size
    dataloader = create_minibatches(X_tensor_train, Y_train_one_hot, batch_size=batch_size)
    for X_tr, y_tr in dataloader:
        # inicializar los gradientes en cero para cada época
        optimizer.zero_grad()
        
        # Propagación hacia adelante
        y_pred = modelo_red_neuronal(X_tr)  #invoca al método forward de la clase MLP
        # Calcular el error MSE
        loss = criterion(y_pred, y_tr)
        #Acumular el error 
        lossTotal += loss.item()
        
        # Propagación hacia atrás: cálculo de los gradientes de los pesos y bias
        loss.backward()
        
        # actualización de los pesos: regla de actualización basado en el gradiente:
        #  
        optimizer.step()
        if np.random.random() < 0.1:
            print(f"Batch Error : {loss.item()}")

    print(f"Época {epoch+1}/{epochs}, Pérdida: {lossTotal/len(dataloader)}")
    
    # Evalúa el modelo con el conjunto de validación
    modelo_red_neuronal.eval()  # Establecer el modo del modelo a "evaluación"
    with torch.no_grad():  # No  calcular gradientes 
        y_pred = modelo_red_neuronal(X_tensor_val)
        # Obtiene una única clase, la más probable
        y_pred = torch.argmax(y_pred, dim=1)
        print(f"Época {epoch+1}/{epochs}")
        print("P=", precision_score(Y_val, y_pred, average='macro'))
        print("R=", recall_score(Y_val, y_pred, average='macro'))
        print("F1=", f1_score(Y_val, y_pred, average='macro'))
        print("Acc=", accuracy_score(Y_val, y_pred))


Iniciando entrenamiento en PyTorch
Batch Error : 1.0525912046432495
Batch Error : 0.9963858723640442
Época 1/30, Pérdida: 1.0407793133155159
Época 1/30
P= 0.495114006514658
R= 0.36082474226804123
F1= 0.26862990810359233
Acc= 0.4984126984126984
Batch Error : 1.011348009109497
Batch Error : 1.0457382202148438
Batch Error : 0.9891188144683838
Batch Error : 0.9728950262069702
Época 2/30, Pérdida: 1.0152599967044333
Época 2/30
P= 0.4405340592390952
R= 0.42076616158121727
F1= 0.37095552681557087
Acc= 0.546031746031746
Batch Error : 0.983300507068634
Batch Error : 0.939422607421875
Batch Error : 0.8298612236976624
Época 3/30, Pérdida: 0.9704341266466223
Época 3/30
P= 0.4206967434815536
R= 0.48430544984893564
F1= 0.4375030841352085
Acc= 0.5936507936507937
Batch Error : 1.034157633781433
Batch Error : 0.9757694005966187
Batch Error : 0.9536284804344177
Época 4/30, Pérdida: 0.939406446788622
Época 4/30
P= 0.7237542331881954
R= 0.489782139074147
F1= 0.4405099971022892
Acc= 0.5841269841269842
Batc

/Users/admin/opt/miniconda3/envs/CE/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/admin/opt/miniconda3/envs/CE/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/admin/opt/miniconda3/envs/CE/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

Batch Error : 0.6867101192474365
Batch Error : 0.8299028873443604
Batch Error : 0.7828238010406494
Batch Error : 0.737697422504425
Época 15/30, Pérdida: 0.7523205980010654
Época 15/30
P= 0.6196171570365119
R= 0.5933094478153575
F1= 0.6020482787403907
Acc= 0.6285714285714286
Batch Error : 0.784403383731842
Batch Error : 0.7127807140350342
Batch Error : 0.7598428726196289
Época 16/30, Pérdida: 0.7230952408002771
Época 16/30
P= 0.621268662997189
R= 0.592171994447436
F1= 0.5991011769603131
Acc= 0.6317460317460317
Batch Error : 0.7311815023422241
Batch Error : 0.8314912915229797
Batch Error : 0.6663540601730347
Batch Error : 0.7674238085746765
Batch Error : 0.6976246237754822
Época 17/30, Pérdida: 0.71991526303084
Época 17/30
P= 0.6236726166144343
R= 0.5765137104410732
F1= 0.5871997413155949
Acc= 0.6253968253968254
Batch Error : 0.6435180306434631
Batch Error : 0.759217381477356
Época 18/30, Pérdida: 0.7212614728056866
Época 18/30
P= 0.6234201691012322
R= 0.586370748295909
F1= 0.59723548860

### Modo para predicción de datos

In [49]:
# TODO: Transformar el dataset de test con los mismos preprocesamientos y al  espacio de 
# representación vectorial que el modelo entrenado, es decir, al espacio de la matriz TFIDF

# Convertir los datos de prueba a tensores de PyTorch

X_tensor_test = torch.from_numpy(X_test)
X_tensor_test = X_tensor_test.to(torch.float32)

# Desactivar el comportamiento de modo de  entrenamiento: por ejemplo, capas como Dropout
modelo_red_neuronal.eval()  # Establecer el modo del modelo a "evaluación"

with torch.no_grad():  # No  calcular gradientes 
    y_pred_test= modelo_red_neuronal(X_tensor_test)

# y_test_pred contiene las predicciones

# Obtener la clase real
y_pred_test = torch.argmax(y_pred_test, dim=1)

print(y_pred_test[:5])


tensor([1, 1, 0, 1, 1])


### Evaluación

In [50]:
# TODO: Evaluar el modelo con las predicciones obtenidas y las etiquetas esperadas: 
# classification_report y  matriz de confusión (métricas Precisión, Recall, F1-measaure, Accuracy)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
Y_test_encoded = le.transform(Y_test)

print(confusion_matrix(Y_test_encoded, y_pred_test))
print(classification_report(Y_test_encoded, y_pred_test, digits=4, zero_division='warn'))


[[ 67  70  36]
 [ 20 307  44]
 [ 21  75 146]]
              precision    recall  f1-score   support

           0     0.6204    0.3873    0.4769       173
           1     0.6792    0.8275    0.7461       371
           2     0.6460    0.6033    0.6239       242

    accuracy                         0.6616       786
   macro avg     0.6485    0.6060    0.6156       786
weighted avg     0.6560    0.6616    0.6492       786



# Guardar el modelo

In [51]:

# Obtener el state_dict (diccionario de pesos y sesgos)
state_dict = modelo_red_neuronal.state_dict()

# Guardar el state_dict
torch.save(state_dict, "./data/red_neuronal_parametros.pth")
